Issues with MADE dataset:
FTIR
* Dodgy sample numbers
* 2 Hit confidence columns
* 2 substance detected columns

In [23]:
# Module imports
import copy
import datetime
import os
import numpy as np
import pandas as pd
from collections import namedtuple

def fix_sample_number(x):
    """Make sure all samples numbers are of form: AXXX (where A is one of A, F, W and X is a digit)"""
    if isinstance(x, float) and np.isnan(x):
        return x # leave NaN's alone    
    try:
        sn = int(x)
        sn = 'F{:04d}'.format(int(x))
    except ValueError:
        # Assume string so make sure it's of the right format
        sn = str(x).capitalize()
    if sn[0] not in ['A', 'F', 'W']:
        print("!!! Bad ID %s" % sn)
    return sn

def now():
    return datetime.datetime.now().strftime("%d/%m/%y %H:%M:%S")

ftir_csv = 'MADE/FTIR Analysis Data Recording Form.csv'
catalog_csv = 'MADE/Sample Cataloguing Form.csv'
reagent_csv = 'MADE/Reagent Outcomes.csv'
hr_csv = 'MADE/MADE MAST Intervention Questionnaire.csv'

date_cols = ['Timestamp']
df_ftir = pd.read_csv(ftir_csv, engine="python", parse_dates=date_cols)
df_catalog = pd.read_csv(catalog_csv, engine="python", parse_dates=date_cols)
df_reagent = pd.read_csv(reagent_csv, engine="python", parse_dates=date_cols)
df_hr = pd.read_csv(hr_csv, engine="python", parse_dates=date_cols)

mla_excel = 'MADE/MADE - Loop 2018 event results sheet_.xlsx'
df_mla = pd.read_excel(mla_excel, sheetname='MLA', header=1)

# Sort out column names
df_reagent.rename(columns={'Sample Code':'Sample Number', 'Substance(s) detected' : 'Reagent Result'}, inplace=True)
df_hr.rename(columns={'Sample Number:':'Sample Number'}, inplace=True)
df_mla.rename(columns={'Sample Num':'Sample Number'}, inplace=True)

# Make all sample numbers a 4-digit code starting with F
df_ftir['Sample Number'] = df_ftir['Sample Number'].apply(fix_sample_number)
df_catalog['Sample Number'] = df_catalog['Sample Number'].apply(fix_sample_number)
df_reagent['Sample Number'] = df_reagent['Sample Number'].apply(fix_sample_number)
df_hr['Sample Number'] = df_hr['Sample Number'].apply(fix_sample_number)
df_mla['Sample Number'] = df_mla['Sample Number'].apply(fix_sample_number)

# Prune down MLA to valid sample numbers
df_mla = df_mla[df_mla['Sample Number'].notnull()]

DataFrames = namedtuple('DataFrames', ['catalog', 'ftir', 'reagent','mla', 'hr'])
dfs = DataFrames(
    catalog=df_catalog,
    ftir=df_ftir,
    reagent=df_reagent,
    mla=df_mla,
    hr=df_hr)

In [94]:
def gsheets_service():
    from googleapiclient.discovery import build
    from httplib2 import Http
    from oauth2client import file, client, tools
    # If modifying these scopes, delete the file token.json.
    CREDS_FILE = '/opt/random/MADE/JensDataExportJupyter_client_secret.json'
    SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        import argparse
        flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args([])
        flow = client.flow_from_clientsecrets(CREDS_FILE, SCOPES)
        creds = tools.run_flow(flow, store, flags)
    service = build('sheets', 'v4', http=creds.authorize(Http()))
    return service

def get_df(service, SPREADSHEET_ID, SS_RANGE, mla=False):
    # Call the Sheets API
    result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID,
                                                range=SS_RANGE).execute()
    values = result.get('values', [])
    if not values:
        print('*** No data found ***')
        return None

    # mla has irrelevant stuff in columns 1 and 3 and sample numbers in first column
    if mla:
        values.pop(0)
        values.pop(1)
        def not_blank(row):
            return len(row[0]) > 0       
    else:
        def not_blank(row):
            return sum(map(len, row[:6])) > 0

    rows = filter(not_blank, values)
    if not rows:
        print('*** No data found after pruning rows! ***')
        return None
    return pd.DataFrame(rows[1:], columns=rows[0])

def canonicalise_df(df, source=None):
    from pandas._libs.tslib import OutOfBoundsDatetime
    """Initial cleaning of all dataframes"""
    df.rename(columns={'Sample Code':'Sample Number',
                       'Sample Number:':'Sample Number',
                       'Sample Num':'Sample Number',
                       'Sample Number i.e F0XXX' : 'Sample Number'
                      }, inplace=True)
    def fix_timestamp(x):
        """Horrror to fix mixed date formats"""
        try:
            x = pd.to_datetime(x)
        except OutOfBoundsDatetime:
            try:
                # '12/08/2018 12:26:15'
                x = pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S')
            except ValueError:
                #'Thu 9/08 - 12:19'
                x = pd.to_datetime(x, format='%a %m/%y - %H:%M')
        return x
    if 'Timestamp' in df.columns:
        df.loc[:, 'Timestamp'] = df['Timestamp'].map(fix_timestamp)
    if source:
        print("Fixing Sample Number for %s" % source)
    df.loc[:, 'Sample Number'] = df['Sample Number'].apply(fix_sample_number)
    df = df[df['Sample Number'].notnull()]
    df.sort_values(['Sample Number'], ascending=True, inplace=True)
    return df
    
# The ID and range of a sample spreadsheet.
BOOMTOWN2018_SPREADSHEET_ID = '1RiA-FwG_954Ger2VPsOSA3JLh-7sEoTYr40eVS0mp24'
MADE2018_SPREADSHEET_ID = '1daXdyL6uL8qnMsEsP0RLZE9nDzt6J7Zr1ygQdguvi-E'
SPREADSHEET_ID = BOOMTOWN2018_SPREADSHEET_ID
CATALOG_RANGE = 'Catalog!A:R'
FTIR_RANGE = 'FTIR!A:X'
REAGENT_RANGE = 'Reagent!A:W'
MLA_RANGE = 'MLA!A:R'
HR_RANGE = 'Interventions!A:BJ'

service = gsheets_service()

df_catalog = get_df(service, SPREADSHEET_ID, CATALOG_RANGE)
df_catalog = canonicalise_df(df_catalog, source='catalog')
df_ftir = get_df(service, SPREADSHEET_ID, FTIR_RANGE)
df_ftir = canonicalise_df(df_ftir, source='ftir')
df_reagent = get_df(service, SPREADSHEET_ID, REAGENT_RANGE)
df_reagent = canonicalise_df(df_reagent, source='reagent')
df_mla = get_df(service, SPREADSHEET_ID, MLA_RANGE, mla=True)
df_mla = canonicalise_df(df_mla, source='mla')
df_hr = get_df(service, SPREADSHEET_ID, HR_RANGE)
df_hr = canonicalise_df(df_hr, source='hr')

DataFrames = namedtuple('DataFrames', ['catalog', 'ftir', 'reagent','mla', 'hr'])
dfs = DataFrames(
    catalog=df_catalog,
    ftir=df_ftir,
    reagent=df_reagent,
    mla=df_mla,
    hr=df_hr)

import pickle
with open('foo.pkl','w') as w:
    pickle.dump(dfs, w)

Fixing Sample Number for catalog
!!! Bad ID R0876
Fixing Sample Number for ftir
!!! Bad ID Z1000
!!! Bad ID B0076
Fixing Sample Number for reagent
!!! Bad ID Z1000
!!! Bad ID Yellow > green
Fixing Sample Number for mla
Fixing Sample Number for hr
!!! Bad ID G0037
!!! Bad ID G0242
!!! Bad ID G0153
!!! Bad ID G0024
!!! Bad ID G0652
!!! Bad ID G0999
!!! Bad ID G0877
!!! Bad ID G0878
!!! Bad ID G0811
!!! Bad ID G1441
!!! Bad ID G1216
!!! Bad ID G1228
!!! Bad ID G1229
!!! Bad ID G1398
!!! Bad ID G1284
!!! Bad ID G1572
!!! Bad ID G0875
!!! Bad ID G1833
!!! Bad ID G1703
!!! Bad ID G1860
!!! Bad ID G1859
!!! Bad ID G1699
!!! Bad ID G1686
!!! Bad ID G1312
!!! Bad ID G0420
!!! Bad ID G0983
!!! Bad ID G0981


In [65]:
# import pickle
# with open('foo.pkl') as f:
#     dfs = pickle.load(f)

# Check for duplicates
if dfs.catalog['Sample Number'].duplicated().any():
    print("### Duplicated catalog sample numbers ###")
    dfs.catalog[dfs.catalog['Sample Number'].duplicated(keep=False)].to_csv('catalog_duplicates.csv')
if dfs.ftir['Sample Number'].duplicated().any():
    print("### Duplicated FTIR sample numbers ###")
    dfs.ftir[dfs.ftir['Sample Number'].duplicated(keep=False)].to_csv('ftir_duplicates.csv')
if dfs.reagent['Sample Number'].duplicated().any():
    print("### Duplicated catalog sample numbers ###")    
    dfs.reagent[dfs.reagent['Sample Number'].duplicated(keep=False)].to_csv('reagent_duplicates.csv')
if dfs.hr['Sample Number'].duplicated().any():
    print("### Duplicated HR sample numbers ###")
    dfs.hr[dfs.hr['Sample Number'].duplicated(keep=False)].to_csv('hr_duplicates.csv', encoding = 'utf-8')
if dfs.mla['Sample Number'].duplicated().any():
    print("### Duplicated MLA sample numbers ###")
    dfs.mla[dfs.mla['Sample Number'].duplicated(keep=False)].to_csv('mla_duplicates.csv')
    
if dfs.catalog['Sample Number'].duplicated().any() or \
    dfs.ftir['Sample Number'].duplicated().any() or \
    dfs.reagent['Sample Number'].duplicated().any() or \
    dfs.hr['Sample Number'].duplicated().any() or \
    dfs.mla['Sample Number'].duplicated().any():

    outs = 'Please fix duplicated values'
    print(outs)
#     raise RuntimeError(outs)

### Duplicated catalog sample numbers ###
### Duplicated FTIR sample numbers ###
### Duplicated catalog sample numbers ###
### Duplicated HR sample numbers ###
Please fix duplicated values


In [95]:
# Check there are no sample numbers in any of the other spreadsheets that aren't in the cataolog sheet
catalog_unique = set(dfs.catalog['Sample Number'].unique())

ftir_unique = set(dfs.ftir['Sample Number'].unique())
ftir_orphan = ftir_unique.difference(catalog_unique)
if ftir_orphan:
    print("Orphaned FTIR sample numbers: %s" % sorted(ftir_orphan))

reagent_unique = set(dfs.reagent['Sample Number'].unique())
reagent_orphan = reagent_unique.difference(catalog_unique)
if reagent_orphan:
    print("Orphaned Reagent Test sample numbers: %s" % sorted(reagent_orphan))

hr_unique = set(dfs.hr['Sample Number'].unique())
hr_orphan = hr_unique.difference(catalog_unique)
if hr_orphan:
    print("Orphaned HR sample numbers: %s" % sorted(hr_orphan))
    
mla_unique = set(dfs.mla['Sample Number'].unique()).difference(catalog_unique)
mla_orphan = mla_unique.difference(catalog_unique)
if mla_orphan:
    print("Orphaned MLA sample numbers: %s" % sorted(mla_orphan))
    
# Check for any that are only in the catalog
outside_catalog = set.union(reagent_unique, hr_unique, mla_unique)
catalog_only = catalog_unique.difference(outside_catalog)
if catalog_only:
    print("Sample numbers only in catalog: %s" % sorted(catalog_only))
    
# Check for any that aren't in FTIR and don't have anything in reagent test
ftir_missing = catalog_unique.difference(ftir_unique).difference(reagent_unique).difference(catalog_only)
if len(ftir_missing):
    print("Samples not in FTIR or Reagent: %s" % sorted(ftir_missing))

all_unique = copy.copy(ftir_unique)
all_unique.update(reagent_unique, hr_unique, mla_unique)
if (all_unique or catalog_only):
    outs = "### Please fix orphaned/catalog only samples ###"
    print(outs)
    #raise RuntimeError(outs)

Orphaned FTIR sample numbers: ['A1439', 'A1904', 'A1910', 'B0076', 'F00129', 'F0054', 'F0057', 'F00742', 'F0156', 'F0225', 'F0272', 'F0285', 'F0343', 'F0378', 'F0571', 'F0795', 'F0876', 'F0883', 'F0967', 'F0983', 'F1179', 'F1197', 'F1201', 'F1277', 'F1413', 'F1518', 'F1878', 'F1880', 'F`1433', 'Z1000']
Orphaned Reagent Test sample numbers: ['F0225', 'F0526', 'F0983', 'F1437', 'F1819 (or 1827?)', 'Yellow > green', 'Z1000']
Orphaned HR sample numbers: ['F0054', 'F0154', 'F0156', 'F0225', 'F0272', 'F0285', 'F0378', 'F0612', 'F0616', 'F0795', 'F0876', 'F0883', 'F0967', 'F1076', 'F1179', 'F1201', 'F1880', 'F2901', 'F9999', 'G0024', 'G0037', 'G0153', 'G0242', 'G0420', 'G0652', 'G0811', 'G0875', 'G0877', 'G0878', 'G0981', 'G0983', 'G0999', 'G1216', 'G1228', 'G1229', 'G1284', 'G1312', 'G1398', 'G1441', 'G1572', 'G1686', 'G1699', 'G1703', 'G1833', 'G1859', 'G1860']
Orphaned MLA sample numbers: ['F0156', 'F0795']
Sample numbers only in catalog: ['F0005', 'F0024', 'F0037', 'F0042', 'F0049', 'F005

In [96]:
# Clean up catalog
# Drop all unwanted columns

#  or 'Your initials'
l = ['Which device was a photo taken with? Who does it belong to?',
     'Is a breakline present?',
     'Unusual appearance'
    ]
i1 = 'Your initials'
i2 = 'Your name and first initial'
if i1 in dfs.catalog.columns:
    l.append(i1)
elif i2 in dfs.catalog.columns:
    l.append(i2)
dfs.catalog.drop(l, axis=1, inplace=True)

d = {
    'Timestamp' : 'Catalog timestamp',
    'Sample Advertised/Acquired/Sold As': 'Catalog Sold As',
    'Sample Form' : 'Catalog form',
    'Has the Service User or a close friend tried this batch?': 'Catalog tried',
    'What is the mass? (mg)': 'Full pill mass',
    'What is the shape of the pill?': 'Pill shape',
    'What is the logo?': 'Pill logo',
    'What colour is the pill?': 'Pill colour'
}
dfs.catalog.rename(columns=d, inplace=True)

In [97]:
# For FTIR columns need to merge the data from the 'Compound detected', 'Hit Confidence.1' columns into the
# 'Substance detected', 'Hit Confidence' column where the substance detected was 'other'
mask = dfs.ftir['Substance detected'] != 'Other'
dfs.ftir['Substance detected'].where(mask, dfs.ftir['Compound detected'], inplace=True) # Copy values from 'Compound detected'
dfs.ftir['Hit Confidence'].where(mask, dfs.ftir['Hit Confidence.1'], inplace=True)
dfs.ftir.drop(['Compound detected', 'Hit Confidence.1', 'Brief Note'], axis=1, inplace=True)

mask = dfs.ftir['Compound detected (Subtraction)'] != 'Other'
dfs.ftir['Compound detected (Subtraction)'].where(mask, df_ftir['Substance detected.1'], inplace=True) # Copy values from 'Compound detected'
dfs.ftir['Hit Confidence.2'].where(mask, dfs.ftir['Hit Confidence.3'], inplace=True)
dfs.ftir.drop(['Substance detected.1', 'Hit Confidence.3', 'Brief Note.1'], axis=1, inplace=True)

# Drop all unwanted columns
l = ['Your name and surname initial',
     'User Suspicion',
     'Is anything detected after subtraction analysis?',
     'Analysis required',
     'Note for harm reduction worker'
    ]
dfs.ftir.drop(l, axis=1, inplace=True)

# Rename shared columns so that we can check for any errors and remove any columns not of interest to the master df
d = {
    'Timestamp' : 'FTIR timestamp',
    'Sample Sold As': 'FTIR Sold As',
    'Sample Form' : 'FTIR form',
    'Has the Service User or a close friend tried this batch?': 'FTIR tried',
    'Substance(s) detected' : 'FTIR final result',
    'Substance detected' : 'FTIR result1',
    'Hit Confidence' :  'FTIR hit1',
    'Is anything detected after subtraction analysis?' : 'FTIR subtraction positive',
    'Compound detected (Subtraction)' :  'FTIR result2',
    'Hit Confidence.2' :  'FTIR hit2',
    '"Strength" of powdered substance' : 'FTIR Powder Strength',
    'Does the substance detected match the substance that was advertised?' : 'FTIR Matches Sold As',
}
dfs.ftir.rename(columns=d, inplace=True)

ValueError: Must specify axis=0 or 1

In [ ]:
# Clean up HR form

# Drop all unwanted columns
l = ['HR worker name:']
dfs.hr.drop(l, axis=1, inplace=True)

# Rename shared columns so that we can check for any errors and remove any columns not of interest to the master df
d = {
    'Timestamp' : 'HR timestamp',
    'You submitted a substance for analysis. What were you told it was when you got it?': 'HR Sold as',
    'Had you already tried this substance before getting it tested?': 'HR tried',
    'What was your first sample number at this event? Did you take a photo or keep the ticket?': 'Previous Sample Number'
}
dfs.hr.rename(columns=d, inplace=True)

In [ ]:
# Catalog and FTIR data frames
df_all = pd.merge(dfs.catalog, dfs.ftir, how='left', on=['Sample Number'])

In [ ]:
# Merge in any reagent test data
df_all = pd.merge(df_all, dfs.reagent[['Sample Number', 'Reagent Result']], how='left', on=['Sample Number'])

In [ ]:
# Merge in any pill strength data
df_all = pd.merge(df_all, dfs.mla[['Sample Number', 'MDMA / tablet (mg)', '% MDMA content']], how='left', on=['Sample Number'])

In [ ]:
# Merge in HR data
df_all = pd.merge(df_all, dfs.hr, how='left', on=['Sample Number'])

In [ ]:
# Fix column orders
prefix = ['Sample Number',
          'Catalog timestamp', 'FTIR timestamp', 'HR timestamp',
          'Catalog Sold As', 'FTIR Sold As','HR Sold as', 
          'Catalog form', 'FTIR form',
          'Catalog tried', 'FTIR tried', 'HR tried']
columns = [c for c in df_all.columns if c not in prefix]
columns = prefix + columns
df_all = df_all[columns]
df_all.to_csv('foo.csv')